#### This sample illustrates multithread decode:
  - 1st thread acts as producer, it reads packets into internal queue.
  - 2nd thread acts as consumer, it takes packets from queue and decodes them to video frames.

In [ ]:
import python_vali as vali
import threading
import time

#### Producer-consumer pattern makes sense when reading input from network. When decoding local files, it doesn't have a great advantage over usual single-thread approach.

In [ ]:
url = "../tests/data/test.mp4"
dec = vali.PyDecoder(url, {}, gpu_id=0)

In [ ]:
def producer() -> None:
    """
    This function reads compressedd video packets.
    It's to be run in separate thread.
    """
    now = time.time()

    while True:
        # This method unlocks GIL, so producer thread may yield to other
        # while being blocked on IO.
        status = dec.ReadPacket()
        if status in [vali.DecodeStatus.OVER, vali.DecodeStatus.ERROR]:
            break

    now = time.time() - now
    print(f"{now:.4f}(s.) spent in producer()")

In [ ]:
def consumer() -> None:
    """
    This function decodes compressed video packets.
    It's to be run in separate thread.
    """
    surf = vali.Surface.Make(dec.Format, dec.Width, dec.Height, gpu_id=0)
    now = time.time()

    while True:
        # This method unlocks GIL, so consumer thread may yield to other.
        status = dec.DecodePacketToSurfaceAsync(surf)
        if status in [vali.DecodeStatus.DONE, vali.DecodeStatus.ERROR]:
            break

    now = time.time() - now
    print(f"{now:.4f}(s.) spent in consumer()")

In [ ]:
producer_th = threading.Thread(target=producer)
consumer_th = threading.Thread(target=consumer)

now = time.time()

producer_th.start()
consumer_th.start()

producer_th.join()
consumer_th.join()

now = time.time() - now
print(f"{now:.4f}(s.) spent in main thread")

In [ ]:
dec = vali.PyDecoder(url, {}, gpu_id=0)
surf = vali.Surface.Make(dec.Format, dec.Width, dec.Height, gpu_id=0)

now = time.time()

while True:
    success, _ = dec.DecodeSingleSurface(surf)
    if not success:
        break

now = time.time() - now
print(f"{now:.4f}(s.) spent in single thread")